# Introduction to Laser Cross Calibration

This tutorial provides a step-by-step introduction to the `laser_cross_calibration` framework. By the end of this tutorial, you will understand:

- What laser cross calibration is and why it is useful
- The core concepts of the framework (optical systems, interfaces, sources, and tracers)
- How to set up a simple optical system
- How to trace laser beams through optical media
- How to validate your simulations against real-world data

## What is Laser Cross Calibration?

Laser cross calibration is a measurement technique where two laser beams intersect to determine the 3D position of their crossing point. By moving a stage with two fixed laser sources and tracking how the intersection point moves, we can calibrate the relationship between stage movement and the measured position.

This technique is particularly useful in:
- 3D metrology and positioning systems
- Optical alignment tasks
- Understanding how light propagates through different media

## Framework Overview

The `laser_cross_calibration` framework simulates how laser beams propagate through optical systems with different materials and geometries. It uses ray tracing to:

1. Track rays through multiple optical interfaces (e.g., air-glass, glass-water boundaries)
2. Apply physics-based refraction at each interface using Snell's law
3. Find where multiple rays intersect in 3D space

## Tutorial Structure

We will work through a real-world example based on research by Gunady et al., who used a 3D printer stage and two laser pointers for calibration. This example demonstrates all key concepts of the framework.

In [ ]:
import laser_cross_calibration as lcc
from math import sin, cos, pi as PI, radians

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress

## Import Required Libraries

We need several libraries for this tutorial:

- `laser_cross_calibration` (imported as `lcc`): The main framework for optical simulation
- `numpy`: Numerical operations and array handling
- `matplotlib`: Plotting and visualization
- `scipy.stats.linregress`: Statistical analysis for validation
- Standard math functions for angle calculations

## Gunady Setup

### Setup
Gunady et al. used a 3D printer xyz-stage and two laser pointer for laser cross calibration. We can use their setup as an example and to validate the simulation pipeline.

![](./images/gunady_setup.png)
Source: [**DOI**: 10.1088/1361-6501/ad574d](https://iopscience.iop.org/article/10.1088/1361-6501/ad574d)

### Logic
The simulation logic works as follows:
1. Create a container for all surface components which can interact with the beams. 
    - The corresponding object is `OpticalSystem`
    - Create surface geometries which define the locations of optical interfaces in space
    - Define optical interfaces from a geometrie and two materials (pre and post surface)
    - Add the interface to the optical system
2. Define a laser beam source
    - In this case we use a `DualLaserStageSource` defining two laser beam sources with fixed geometrical relations
3. Create a tracer (`RayTracer`)
    - The tracer takes the system and a list of source to trace the rays emitted by the sources through the optical system.
4. Optionally the scene can be visualized using a `Scene` object.
    - Add all components to visualize and use the `make_figure` method to create a `plotly` graph


### Sketch
![](./images/gunady_setup_sketch.png)

The angles are `alpha = 11.5 deg` and `beta = 12.6 deg` according to Gunady et al.

### Logic Pipeline

## Core Concepts

Before building the optical system, let's understand the key concepts:

### 1. OpticalSystem
The `OpticalSystem` is a container that holds all optical interfaces (surfaces) that interact with light rays. Think of it as your virtual optical bench where you place all components.

**Key parameter:**
- `final_propagation_distance`: After passing through all interfaces, rays continue propagating this distance in the final medium. This ensures rays travel far enough to find intersections.

### 2. Optical Interfaces
An `OpticalInterface` represents a boundary between two materials where refraction occurs (e.g., an air-glass surface). Each interface consists of:

- **Geometry**: The shape and position of the surface (e.g., plane, sphere)
- **material_pre**: The material BEFORE the surface (where the ray comes from)
- **material_post**: The material AFTER the surface (where the ray goes to)

### 3. Surface Normals
Each surface has a **normal vector** that points OUTWARD from the material:
- The normal defines which side is "inside" vs "outside" the material
- When a ray hits the surface, the framework uses the normal to determine the angle of incidence
- **Critical**: Always ensure normals point outward from solid materials

### 4. Materials
The framework includes predefined materials with their refractive indices:
- `lcc.materials.AIR`: Refractive index ~1.0
- `lcc.materials.WATER`: Refractive index ~1.33
- `lcc.materials.GLASS_FUSED_SILICA`: Refractive index ~1.46

### 5. Sources
Sources define where laser beams originate and their direction. The `DualLaserStageSource` simulates a stage with two laser pointers at fixed positions and angles.

### 6. RayTracer
The `RayTracer` performs the actual simulation:
- Takes rays from sources
- Propagates them through the optical system
- Applies refraction at each interface
- Finds where rays intersect

In [ ]:
import mermaid as md
from mermaid.graph import Graph

render = md.Mermaid("""
graph LR
    Source(("Source<br/>(Ray Origin)"))
    Air["Air<br/>(Ray Propagation)"]
    FrontSurface{"Front Surface<br/>(Air-Glass Interface)<br/>Refraction"}
    Glass["Glass<br/>(Ray Propagation)"]
    BackSurface{"Back Surface<br/>(Glass-Water Interface)<br/>Refraction"}
    Water["Water<br/>(Ray Propagation)"]
    Intersection(("Intersection"))
                    
                    
    Source -->|"Ray"| Air
    Air -->|"Ray"| FrontSurface
    FrontSurface -->|"Ray"| Glass
    Glass -->|"Ray"| BackSurface
    BackSurface -->|"Ray"| Water     
    Water --> Intersection   
                    
    style Source fill:#008cff
    style Intersection fill:#1e8604
    style FrontSurface fill: #ffaa00
    style BackSurface fill: #ffaa00
""")

c = "#1e8604"
render

### Create Optical System

Now we build the optical system step by step. Our setup consists of:
- A glass plate (1 cm thick) submerged in water
- Laser beams starting in air, passing through glass, and ending in water

This creates two optical interfaces:
1. Air-Glass interface (front surface)
2. Glass-Water interface (back surface)

In [ ]:
# Create an optical system to hold all interfaces
# final_propagation_distance: rays continue 10 meters after the last interface
system = lcc.tracing.OpticalSystem(final_propagation_distance=10)

# FRONT SURFACE: Air-Glass interface at y=0
# The surface is located at point (0,0,0)
# Normal vector points in -y direction (outward from glass into air)
plane_front_surface = lcc.surfaces.Plane(
    point=lcc.constants.UNIT_Y_VECTOR3 * 0, 
    normal=-lcc.constants.UNIT_Y_VECTOR3
)

# Create the interface: rays come from air and enter glass
plane_front_interface = lcc.tracing.OpticalInterface(
    geometry=plane_front_surface,
    material_pre=lcc.materials.AIR,        # ray comes from air
    material_post=lcc.materials.GLASS_FUSED_SILICA  # ray enters glass
)

# Add the front interface to our system
system.add_interface(plane_front_interface)

# BACK SURFACE: Glass-Water interface at y=0.01 (1 cm thickness)
# The surface is located at point (0, 0.01, 0)
# Normal vector points in +y direction (outward from glass into water)
# IMPORTANT: The normal direction is opposite to the front surface!
plane_back_surface = lcc.surfaces.Plane(
    point=lcc.constants.UNIT_Y_VECTOR3 * 1e-2, 
    normal=lcc.constants.UNIT_Y_VECTOR3  # normal points OUT of glass
)

# Create the interface: rays come from glass and enter water
plane_back_interface = lcc.tracing.OpticalInterface(
    geometry=plane_back_surface,
    material_pre=lcc.materials.GLASS_FUSED_SILICA,  # ray comes from glass
    material_post=lcc.materials.WATER                # ray enters water
)

# Add the back interface to our system
system.add_interface(plane_back_interface)

### Visualize the Optical System

Let's visualize what we just created. The `Scene` object provides interactive 3D visualization using plotly.

We can see:
- The two parallel surfaces (glass plate boundaries)
- The coordinate system orientation

In [ ]:
# Create a 3D visualization scene
scene = lcc.visualization.Scene()

# Define the volume boundaries for the scene (in meters)
scene.bounds_max = (0.5, 0.5, 0.5)
scene.bounds_min = (-0.5, -0.5, -0.5)

# Add our optical system to the scene
scene.add_system(system)

# Create an interactive plotly figure
# display_bounds_* controls the zoomed view (smaller than scene bounds)
scene.make_figure(
    display_bounds_max=(0.1, 0.1, 0.1), 
    display_bounds_min=(-0.1, -0.1, -0.1)
)

### Create the Laser Source

Now we create a dual laser source that mimics the Gunady et al. experimental setup:
- Two laser pointers mounted on a stage
- Separated by 20 cm
- Each pointing at specific angles (11.5° and 12.6°)
- Stage positioned 30 cm away the glass plate

In [ ]:
# Position the stage 30 cm away the glass plate (negative y direction)
position = lcc.constants.ORIGIN_POINT3 - lcc.constants.UNIT_Y_VECTOR3 * 0.3

# Distance between the two laser pointers on the stage
distance_between_sources = 0.2  # 20 cm

# Beam angles from the Gunady et al. publication
alpha = radians(11.5)  # left laser angle
beta = radians(12.6)   # right laser angle

# Calculate the direction vectors for each laser beam
# Left beam points up-right: negative x component, positive y component
direction_beam1 = np.array((-sin(alpha), cos(alpha), 0.0))
# Right beam points up-left: positive x component, positive y component
direction_beam2 = np.array((sin(beta), cos(beta), 0.0))

# Create the dual laser source
# arm1 and arm2 define the positions of each laser relative to the origin
source = lcc.sources.DualLaserStageSource(
    origin=position,  # center of the stage
    arm1=lcc.constants.UNIT_X_VECTOR3 * distance_between_sources / 2,   # right laser (+x)
    arm2=-lcc.constants.UNIT_X_VECTOR3 * distance_between_sources / 2,  # left laser (-x)
    direction1=direction_beam1,  # direction of beam from arm1
    direction2=direction_beam2,  # direction of beam from arm2
)

### Visualize System and Source Together

Now let's visualize both the optical system and the laser source. The lasers are shown as colored cones pointing in the beam direction.

In [ ]:
# Clear any previously added sources and add our new source
scene.clear_sources()
scene.add_source(source)

# Render the scene: sources appear as colored cones indicating beam direction
scene.make_figure()

### Trace Rays and Find Intersection Point

Now comes the main simulation step:
1. Create a `RayTracer` and add our optical system
2. Trace rays from the source through all interfaces
3. Find where the two beams intersect in 3D space
4. Visualize the complete scene with ray paths and intersection point

In [ ]:
# Create a ray tracer and register our optical system
tracer = lcc.tracing.RayTracer()
tracer.add_optical_system(system=system)

# Trace rays through the system and find where they cross
# Returns:
#   - rays: list of Ray objects containing the full path through all media
#   - intersection: 3D coordinates where the beams intersect
rays, intersection = tracer.trace_and_find_crossings(sources=[source])

# Visualize the traced rays
scene.clear_rays()
scene.add_rays(rays=rays)

# Add the intersection point if found
if intersection:
    x, y, z = intersection[0]
    scene.clear_points()
    scene.add_point(x=x, y=y, z=z)

# Display the complete scene: system + sources + rays + intersection
scene.make_figure()

### Validate System Against Published Data

Now we validate our simulation against real experimental data. Gunady et al. reported that when they moved their stage by a certain distance in the y-direction, the intersection point moved by **1.34 times** that distance.

We will:
1. Move the stage in steps and record intersection positions
2. Create a 2D visualization showing ray paths and intersection movement
3. Perform linear regression to extract the scaling factor
4. Compare our simulated factor with the published value

#### Simulate Stage Movement

In [ ]:
# Save the original position so we can reset after the simulation
original_position = source.origin.copy()

# Storage for results
intersections = []
rays_list = []
stage_shifts = []

# Define the movement parameters
total_shift = 0.25      # move stage 25 cm total
number_of_steps = 10    # in 10 discrete steps
step_dy = total_shift / number_of_steps

# Simulate stage movement
for i in range(number_of_steps):
    # Trace rays and find intersection at current position
    rays, intersection = tracer.trace_and_find_crossings(sources=[source])
    
    # Store results
    stage_shifts.append(i * step_dy)
    rays_list.append(rays)
    intersections.append(intersection[0])
    
    # Move the stage upward by one step (positive y direction)
    source.translate(y=step_dy)

# Reset stage to original position (important for re-running this cell)
source.origin = original_position

#### Visualize Stage Movement

This 2D plot (x-y plane) shows:
- **Gray region**: The glass plate
- **Red/Blue lines**: Ray paths from the two lasers at different stage positions
- **Orange dashed lines**: Stage y-positions (getting lighter with each step)
- **Green dashed lines**: Intersection y-positions (getting lighter with each step)
- **Black circles**: Intersection points

Notice how the intersection points move more than the stage positions due to refraction!

In [ ]:
fig, ax = plt.subplots()

ax.set_xlabel("y coordinate")
ax.set_ylabel("x coordinate")
ax.set_xlim(-0.35, 0.55)
ax.set_ylim(-0.2, 0.2)

# Draw the glass plate as a gray rectangle
ax.axvspan(
    plane_front_surface.point[1],
    plane_back_surface.point[1],
    alpha=0.8,
    facecolor="gray",
    edgecolor="black",
)

# Create color gradients for the two laser beams
colors1 = plt.cm.Reds(np.linspace(0.2, 0.8, len(rays_list)))
colors2 = plt.cm.Blues(np.linspace(0.2, 0.8, len(rays_list)))

# Plot all ray paths and intersections
for rays, intersection, color1, color2 in zip(
    rays_list, intersections, colors1, colors2, strict=True
):
    # Plot ray paths (one red, one blue)
    for ray, color in zip(rays, (color1, color2), strict=True):
        x, y, z = list(zip(*ray.path_positions, strict=True))
        ax.plot(y, x, c=color, marker="o", markersize=2.5, markeredgecolor="black")

    # Mark stage y-position with orange dashed line
    ax.axvline(rays[-1].path_positions[0][1], ls="--", c="orange", alpha=0.5)

    # Mark intersection point with black circle
    ax.scatter(
        intersection[1], intersection[0], marker="o", ec="black", fc="none", zorder=10
    )
    
    # Mark intersection y-position with green dashed line
    ax.axvline(intersection[1], ls="--", c="forestgreen", alpha=0.5)

# Add labels
ax.text(
    0.2,
    0.9,
    "Stage Movement",
    transform=ax.transAxes,
    ha="center",
    bbox={"facecolor": "white"},
)
ax.text(
    1 - 0.2,
    0.9,
    "Intersection Movement",
    transform=ax.transAxes,
    ha="center",
    bbox={"facecolor": "white"},
);

#### Calculate Scaling Factor via Linear Regression

To extract the scaling factor quantitatively, we plot stage displacement vs. intersection displacement and fit a linear model.

The **slope** of this line is the scaling factor we're looking for. If the simulation is correct, it should match the published value of **1.34**.

In [ ]:
# Convert intersections to numpy array for easier processing
intersection_array = np.array(intersections)

# Calculate the shift of intersection points relative to the first position
# intersection_array.T[1] extracts all y-coordinates
intersection_shifts = intersection_array.T[1] - intersection_array.T[1][0]

# Create the plot
fig, ax = plt.subplots()
ax.grid()

# Plot simulated data points
ax.scatter(
    np.array(stage_shifts) / step_dy,  # normalize to step units
    intersection_shifts / step_dy,      # normalize to step units
    marker="o",
    ec="black",
    fc="none",
    label="simulation",
)
ax.set_xlabel("stage shift")
ax.set_ylabel("intersection shift")

# Perform linear regression to find the scaling factor
result = linregress(stage_shifts, intersection_shifts)

# Plot the fitted line
ax.axline(
    xy1=(0, result.intercept),
    slope=result.slope,
    c="red",
    ls="--",
    label="linear regression",
)

# Display the results
ax.text(
    0.05,
    0.95,
    f"slope: {result.slope:.3f}\nR: {result.rvalue:.3f}",
    transform=ax.transAxes,
    ha="left",
    va="top",
    bbox={"facecolor": "white"},
)

ax.legend();

### Results

The obtained slope is **1.343**, which excellently matches the published value of **1.34** from Gunady et al.

This validates that our simulation correctly models:
- Ray refraction at optical interfaces
- Propagation through multiple media
- The geometric relationship between stage movement and intersection point displacement

The high R-value (close to 1.0) confirms the linear relationship expected for this simple optical system.

## Summary and Next Steps

### What You Learned

In this tutorial, you learned the fundamental workflow of the `laser_cross_calibration` framework:

1. **Building Optical Systems**: Creating interfaces with proper geometries, materials, and normal vectors
2. **Defining Sources**: Setting up laser beam sources with positions and directions
3. **Ray Tracing**: Using the `RayTracer` to simulate beam propagation through optical media
4. **Finding Intersections**: Calculating where multiple beams cross in 3D space
5. **Validation**: Comparing simulation results with experimental data

### Key Takeaways

- **Normal vectors must point outward** from materials for correct refraction calculations
- The framework handles complex material transitions automatically using Snell's law
- `Scene` objects provide powerful 3D visualization for understanding your setup
- The simulation accurately reproduces real-world optical behavior

### Next Steps

Now that you understand the basics, you can:

1. **Experiment with this setup**:
   - Try different glass thicknesses
   - Change the laser angles
   - Use different materials (try different types of glass)
   - Add more interfaces (multiple glass plates)

2. **Explore more complex geometries**:
   - Use curved surfaces (`lcc.surfaces.Sphere`, `lcc.surfaces.Cylinder`)
   - Create custom geometries
   - Build multi-element optical systems

3. **Advanced analysis**:
   - Study non-linear relationships in complex systems
   - Analyze multiple crossing points
   - Optimize optical configurations

4. **Check the documentation**:
   - API reference for all available materials and surfaces
   - Advanced raytracing options
   - Custom material definitions

### Additional Resources

- **Original publication**: [Gunady et al. (2024), DOI: 10.1088/1361-6501/ad574d](https://iopscience.iop.org/article/10.1088/1361-6501/ad574d)
- **Framework documentation**: Check the project README and API docs
- **Example notebooks**: Explore other examples in the `examples/` directory